In [194]:
import numpy as np
import math
from PIL import Image  # for creating visual of our env
import cv2  # for showing our visual live

In [195]:
class agent:
    def __init__(self,start):
        self.x = start[0]
        self.y = start[1]
    
    def setit(self,point):
        self.x = point[0]
        self.y = point[1]

    def action(self, choice):
        '''
        Gives us 4 total movement options. (0,1,2,3)
        '''
        if choice == 0: #up
            self.move(x=-1, y=0)
        elif choice == 1: #down
            self.move(x=1, y=0)
        elif choice == 2: #right
            self.move(x=0, y=1)
        elif choice == 3: #left
            self.move(x=0, y=-1)

    def move(self, x, y):
        self.x += x
        self.y += y


        #checking for out of bounds
        if self.x < 0:
            self.x = 0
        elif self.x > SIZE-1:
            self.x = SIZE-1
        if self.y < 0:
            self.y = 0
        elif self.y > SIZE-1:
            self.y = SIZE-1

In [196]:
f = open("text.txt", "r")
SIZE=12
grid=np.zeros((SIZE,SIZE))
count=0
lines = f.read().splitlines()
for line in lines:
    #print(line)
    grid[count]=line.split(' ')
    count=count+1
f.close()
print(grid)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0.]
 [0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0.]
 [0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [197]:
st=[]
en=[]
g = open("points.txt","r")
lins=g.read().splitlines()
for lin in lins:
    temp=lin.split(' ')
    st.append([int(temp[0]),int(temp[1])])
    en.append([int(temp[2]),int(temp[3])])
print(st)
print(en)
leng = len(st)
points = []
for w in range(0,leng):
    points.append([st[w],en[w]])
points.sort(key = lambda p : abs(p[0][0]-p[1][0]) + abs(p[0][1]-p[1][1]))
print(points)
for absi in range(0,leng):
    st[absi] = points[absi][0]
    en[absi] = points[absi][1]
print(st)
print(en)
    

[[11, 0], [0, 0], [0, 3], [6, 0], [8, 8]]
[[9, 8], [1, 11], [6, 3], [6, 11], [10, 11]]
[[[8, 8], [10, 11]], [[0, 3], [6, 3]], [[11, 0], [9, 8]], [[6, 0], [6, 11]], [[0, 0], [1, 11]]]
[[8, 8], [0, 3], [11, 0], [6, 0], [0, 0]]
[[10, 11], [6, 3], [9, 8], [6, 11], [1, 11]]


In [198]:
q_table=[]
for i in range(0,len(st)):
    q_table.append(dict())
    
#initializing q_table
for n in range(0,len(st)):
    for x in range(0,SIZE):
        for y in range(0,SIZE):
            for a in range(0,4):
                q_table[n][((x,y),a)]=0

In [199]:
no_of_ep=500
epsilon=1
decay=0.998
learning_rate=0.1
discount_factor=0.8
lmbda=0.9

In [200]:
#trainings

for n in range(0,len(st)):
    for episode in range(0,500):
        ag=agent(start=st[n])
        elgi={}
        for x in range(0,SIZE):
            for y in range(0,SIZE):
                for a in range(0,4):
                    elgi[((x,y),a)]=0
        ep_rew=0
        for step in range(0,100):
            current=(ag.x,ag.y)
            if np.random.random()>epsilon: #exploitation
                action=np.argmax([q_table[n][(current,0)],q_table[n][(current,1)],q_table[n][(current,2)],q_table[n][(current,3)]])
            else: #exploration
                action=np.random.randint(0,3)
            ag.action(action)
            new_s=(ag.x,ag.y)
            #identify reward
            if(grid[ag.x][ag.y]==1): #obstacle
                rew=-5
            elif(ag.x==en[n][0] and ag.y==en[n][1]): #destination
                rew=5
            else: #normal free cell
                rew=-3
            
            max_fut_q=max([q_table[n][(new_s,0)],q_table[n][(new_s,1)],q_table[n][(new_s,2)],q_table[n][(new_s,3)]])    
            delta=rew+(discount_factor*max_fut_q)-q_table[n][(current,action)]
            elgi[(current,action)]+=1
        
            if new_s[0]==current[0] and new_s[1]==current[1]:
                q_table[n][(current,action)]=-50
                continue
            if rew==-5:
                q_table[n][(current,action)]=-50
                ag.setit(point=current)
                continue
            #updating q table
            for x in range(0,SIZE):
                for y in range(0,SIZE):
                    for a in range(0,4):
                        q_table[n][((x,y),a)]= q_table[n][((x,y),a)]+(learning_rate*delta*elgi[((x,y),a)])
                        elgi[((x,y),a)]=elgi[((x,y),a)]*discount_factor*lmbda
            ep_rew+=rew
        
            if rew==5:
                break
        epsilon*=decay
        

In [201]:
occupied_loc=[]
for i in range(0,70):
    occupied_loc.append(set())

In [202]:
def checkfun(new,time):
    neighbours=[(0,1),(1,0),(0,-1),(-1,0),(1,-1),(-1,1),(1,1),(-1,-1)]
    checklist=set()
    for item in neighbours:
        checklist.add((new[0]+item[0],new[1]+item[1]))
    for item in checklist:
        if item in occupied_loc[time]:
            return True
    return False

In [203]:
li=[]
for n in range(0,len(st)):
    li.append(list())

for n in range(0,len(st)):
    age=agent(start=st[n])
    li[n].append(tuple(st[n]))
    for tim in range(1,70):
        current=(age.x,age.y)
        l=[(q_table[n][(current,0)],0),(q_table[n][(current,1)],1),(q_table[n][(current,2)],2),(q_table[n][(current,3)],3)]
        l.sort()
        l.reverse()
        actionset=[l[0][1],l[1][1],l[2][1],l[3][1]]
        action=actionset[0]
        age.action(action)
        new=(age.x,age.y)
        ind=1
        while ((new in occupied_loc[tim]) or checkfun(new=new,time=tim)) and ind<4:
            age.setit(point=current)
            action=actionset[ind]
            age.action(action)
            new=(age.x,age.y)
            ind=ind+1
        
        li[n].append(new)
        occupied_loc[tim].add(new)
        if new[0]==en[n][0] and new[1]==en[n][1]:
            break
            
            
for i in range(0,len(st)):
    print(li[i])
        

[(8, 8), (9, 8), (10, 8), (10, 9), (10, 10), (10, 11)]
[(0, 3), (1, 3), (2, 3), (3, 3), (4, 3), (5, 3), (6, 3)]
[(11, 0), (11, 1), (11, 2), (11, 3), (11, 4), (10, 4), (10, 5), (10, 6), (10, 7), (10, 8), (9, 8)]
[(6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (6, 8), (6, 9), (6, 10), (6, 11)]
[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (1, 11)]


In [208]:
arr_time=0
for l in li:
    if len(l)>arr_time:
        arr_time=len(l)
print(arr_time)

13


In [209]:
newset= set()
for l in li:
    for item in l:
        newset.add(item)
used_cells=len(newset)
print(used_cells)

45


In [206]:
env=np.zeros((SIZE,SIZE,3),dtype=np.uint8)
for x in range(0,SIZE):
    for y in range(0,SIZE):
        if(grid[x][y]==0):
            env[x][y]=(255,255,255)
temp=[(255,255,0),(0,255,255),(127,255,0),(255,0,255),(192,192,192),(128,0,0),(128,0,128)]            
for p in range(0,len(li)):
    tr=temp[p]
    for point in li[p]:
        env[point[0]][point[1]]=tr
        
img = Image.fromarray(env, 'RGB')  # reading to rgb
img = img.resize((300, 300),resample=Image.NEAREST)  # resizing so we can see our agent
cv2.imshow("image", np.array(img))  # show it
cv2.waitKey(0)

-1

In [207]:
img = img.save("output.png")